In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
def getTitle(title,count,uparticle):
    title_templist=list()
    for s in reversed(title):
        title_templist.append(s.text)
    if count==0:                 #第一頁的置頂貼文不是我們要的，根據每個版的置頂貼文數量不同，刪掉不同數量的文章。
        title_templist=title_templist[uparticle:]
        
    return title_templist

In [3]:
def getUrl(title,count,uparticle):
    url_templist=list()
    for t in reversed(title):
        url_templist.append(t['href'])
    if count==0:
        url_templist=url_templist[uparticle:]
        
    return url_templist

In [4]:
def getContent(r):
    
    a=r.text
    b=a.split('article-metaline')[4]
    c=b.split('</span></div>')[1]
    d=c.split('<span class=')[0]
    
    return d

In [5]:
def getAuthor(meta):
    author=meta[0].text
    
    return author

In [6]:
def getTime(meta):
    time=meta[3].text
    
    return time

In [7]:
url = "https://www.ptt.cc/bbs/MobileComm/index.html"

titlelist=list()
urllist=list()
contentlist=list()
timelist=list()
authorlist=list()

uparticle=5 #輸入有幾篇置頂文章
count=0
for i in range(2): #往上爬?頁
    r = requests.get(url)
    soup = BeautifulSoup(r.text,"html.parser")
    title = soup.select("div.title a") #標題
    
    titlelist=titlelist+getTitle(title, count, uparticle)
    urllist=urllist+getUrl(title, count, uparticle)
    
    u = soup.select("div.btn-group.btn-group-paging a") #a標籤
    url = "https://www.ptt.cc"+ u[1]["href"] #上一頁的網址
    
    count+=1
for j in range(len(urllist)):
    c_url='https://www.ptt.cc'+urllist[j]
    r=requests.get(c_url)
    contentlist.append(getContent(r))
    
    soup=BeautifulSoup(r.text, 'html.parser')
    meta=soup.select("span.article-meta-value")
    authorlist.append(getAuthor(meta))
    timelist.append(getTime(meta))

In [8]:
s1 = pd.Series(titlelist, name='title')
s2 = pd.Series(authorlist, name='author')
s3 = pd.Series(timelist, name='time')
s4 = pd.Series(contentlist, name='content')



In [9]:
crawltable = pd.concat([s1, s2, s3, s4], axis=1)

In [10]:
crawltable.head(4)

,title,author,time,content
0,[問題] 小米選擇,mizubishi (mizubishi),Mon Dec 9 20:56:04 2019,\n之前的安卓機快掛了\n差不多可以換一台\n\n原本看上VIVO的Y17\n出門看了很多地...
1,[問題] LM211+PIXEL 4 XL 沒有LTE+？,sdtt (飛鑽夜),Mon Dec 9 20:50:06 2019,\n先前有看過遠傳本來沒有，後續更新後就有了，想說lm應該同是遠傳，也會改正吧…\n\n忍到...
2,[新聞] 又一個應用程式終結！ Windows 10 Mobile,olmtw (支持htc，支持台灣貨),Mon Dec 9 20:33:00 2019,\n
3,[情報] pc家realme 5 pro逼近名店價,bigbowl ( Gathering Storm。),Mon Dec 9 20:24:45 2019,\n呃…我最近好像pc家/realme業配喔\n\n名店5 pro最低看到現金價6500\n...


In [11]:
crawltable.to_csv('pttcrawl.csv')